# Self-Attention and Positional Encoding
:label:`sec_self-attention-and-positional-encoding`

In deep learning,
we often use CNNs or RNNs to encode a sequence.
Now with attention mechanisms.
imagine that we feed a sequence of tokens
into attention pooling
so that
the same set of tokens
act as queries, keys, and values.
Specifically,
each query attends to all the key-value pairs
and generates one attention output.
Since the queries, keys, and values
come from the same place,
this performs
*self-attention* :cite:`Lin.Feng.Santos.ea.2017,Vaswani.Shazeer.Parmar.ea.2017`, which is also called *intra-attention* :cite:`Cheng.Dong.Lapata.2016,Parikh.Tackstrom.Das.ea.2016,Paulus.Xiong.Socher.2017`.
In this section,
we will discuss sequence encoding using self-attention,
including using additional information for the sequence order.


In [1]:
%use @file[../djl-pytorch.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
//import jp.live.ugai.d2j.attention.Chap10Utils
import jp.live.ugai.d2j.attention.MultiHeadAttention
// %load ../utils/attention/MultiHeadAttention.java
// %load ../utils/attention/PositionalEncoding.java
// %load ../utils/attention/Chap10Utils.java
// %load ../utils/attention/DotProductAttention.java
// %load ../utils/attention/MultiHeadAttention.java
// %load ../utils/attention/PositionalEncoding.java

In [2]:
val manager = NDManager.newBaseManager()

## Self-Attention

Given a sequence of input tokens
$\mathbf{x}_1, \ldots, \mathbf{x}_n$ where any $\mathbf{x}_i \in \mathbb{R}^d$ ($1 \leq i \leq n$),
its self-attention outputs
a sequence of the same length
$\mathbf{y}_1, \ldots, \mathbf{y}_n$,
where

$$\mathbf{y}_i = f(\mathbf{x}_i, (\mathbf{x}_1, \mathbf{x}_1), \ldots, (\mathbf{x}_n, \mathbf{x}_n)) \in \mathbb{R}^d$$

according to the definition of attention pooling $f$ in
:eqref:`eq_attn-pooling`.
Using multi-head attention,
the following code snippet
computes the self-attention of a tensor
with shape (batch size, number of time steps or sequence length in tokens, $d$).
The output tensor has the same shape.


In [3]:
    val numHiddens = 100
    val numHeads = 5
    val attention = MultiHeadAttention(numHiddens, numHeads, 0.5f, false)


In [4]:
    val batchSize = 2
    val numQueries = 4
    val validLens = manager.create(floatArrayOf(3.0f, 2.0f))
    var X = manager.ones(Shape(batchSize.toLong(), numQueries.toLong(), numHiddens.toLong()))
    val ps = ParameterStore(manager, false)
    var input = NDList(X, X, X, validLens)
    attention.initialize(manager, DataType.FLOAT32, *input.shapes)
    val result = attention.forward(ps, input, false)
    println(result.get(0).shape)

(2, 4, 100)


## Comparing CNNs, RNNs, and Self-Attention
:label:`subsec_cnn-rnn-self-attention`

Let us
compare architectures for mapping
a sequence of $n$ tokens
to another sequence of equal length,
where each input or output token is represented by
a $d$-dimensional vector.
Specifically,
we will consider CNNs, RNNs, and self-attention.
We will compare their
computational complexity, 
sequential operations,
and maximum path lengths.
Note that sequential operations prevent parallel computation,
while a shorter path between
any combination of sequence positions
makes it easier to learn long-range dependencies within the sequence :cite:`Hochreiter.Bengio.Frasconi.ea.2001`.


![Comparing CNN (padding tokens are omitted), RNN, and self-attention architectures.](https://raw.githubusercontent.com/d2l-ai/d2l-en/master/img/cnn-rnn-self-attention.svg)
:label:`fig_cnn-rnn-self-attention`

Consider a convolutional layer whose kernel size is $k$.
We will provide more details about sequence processing
using CNNs in later chapters.
For now,
we only need to know that
since the sequence length is $n$,
the numbers of input and output channels are both $d$,
the computational complexity of the convolutional layer is $\mathcal{O}(knd^2)$.
As :numref:`fig_cnn-rnn-self-attention` shows,
CNNs are hierarchical  so 
there are $\mathcal{O}(1)$ sequential operations
and the maximum path length is $\mathcal{O}(n/k)$.
For example, $\mathbf{x}_1$ and $\mathbf{x}_5$
are within the receptive field of a two-layer CNN
with kernel size 3 in :numref:`fig_cnn-rnn-self-attention`.

When updating the hidden state of RNNs,
multiplication of the $d \times d$ weight matrix
and the $d$-dimensional hidden state has 
a computational complexity of $\mathcal{O}(d^2)$.
Since the sequence length is $n$,
the computational complexity of the recurrent layer
is $\mathcal{O}(nd^2)$.
According to :numref:`fig_cnn-rnn-self-attention`,
there are $\mathcal{O}(n)$ sequential operations
that cannot be parallelized
and the maximum path length is also $\mathcal{O}(n)$.

In self-attention,
the queries, keys, and values 
are all $n \times d$ matrices.
Consider the scaled dot-product attention in
:eqref:`eq_softmax_QK_V`,
where a $n \times d$ matrix is multiplied by
a $d \times n$ matrix,
then the output $n \times n$ matrix is multiplied
by a $n \times d$ matrix.
As a result,
the self-attention
has a $\mathcal{O}(n^2d)$ computational complexity.
As we can see in :numref:`fig_cnn-rnn-self-attention`,
each token is directly connected
to any other token via self-attention.
Therefore,
computation can be parallel with $\mathcal{O}(1)$ sequential operations
and the maximum path length is also $\mathcal{O}(1)$.

All in all,
both CNNs and self-attention enjoy parallel computation
and self-attention has the shortest maximum path length.
However, the quadratic computational complexity with respect to the sequence length
makes self-attention prohibitively slow for very long sequences.





## Positional Encoding
:label:`subsec_positional-encoding`


Unlike RNNs that recurrently process
tokens of a sequence one by one,
self-attention ditches
sequential operations in favor of 
parallel computation.
To use the sequence order information,
we can inject
absolute or relative
positional information
by adding *positional encoding*
to the input representations.
Positional encodings can be 
either learned or fixed.
In the following, 
we describe a fixed positional encoding
based on sine and cosine functions :cite:`Vaswani.Shazeer.Parmar.ea.2017`.

Suppose that
the input representation $\mathbf{X} \in \mathbb{R}^{n \times d}$ contains the $d$-dimensional embeddings for $n$ tokens of a sequence.
The positional encoding outputs
$\mathbf{X} + \mathbf{P}$
using a positional embedding matrix $\mathbf{P} \in \mathbb{R}^{n \times d}$ of the same shape,
whose element on the $i^\mathrm{th}$ row 
and the $(2j)^\mathrm{th}$
or the $(2j + 1)^\mathrm{th}$ column is

$$\begin{aligned} p_{i, 2j} &= \sin\left(\frac{i}{10000^{2j/d}}\right),\\p_{i, 2j+1} &= \cos\left(\frac{i}{10000^{2j/d}}\right).\end{aligned}$$
:eqlabel:`eq_positional-encoding-def`

At first glance,
this trigonometric-function
design looks weird.
Before explanations of this design,
let us first implement it in the following `PositionalEncoding` class.


In [5]:
class PositionalEncoding(numHiddens: Int, dropout: Float, maxLen: Int, manager: NDManager) : AbstractBlock() {
    private val dropout: Dropout
    var P: NDArray

    init {
        this.dropout = Dropout.builder().optRate(dropout).build()
        addChildBlock("dropout", this.dropout)

        // Create a long enough `P`
        P = manager.zeros(Shape(1, maxLen.toLong(), numHiddens.toLong()))
        val X = manager.arange(maxLen)
            .reshape(-1, 1)
            .div(
                manager.create(10000)
                    .pow(manager.arange(0, numHiddens, 2).div(numHiddens))
            )
        P[NDIndex(":, :, {}::{}", 0, 2)] = X.sin()
        P[NDIndex(":, :, {}::{}", 1, 2)] = X.cos()
    }

    override fun forwardInternal(
        parameterStore: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        var X = inputs[0]
        X = X.add(P[":, :{}, :", X.shape[1]])
        return NDList(
            dropout.forward(parameterStore, NDList(X), training, params)[0]
        )
    }

    override fun getOutputShapes(inputShapes: Array<Shape>): Array<Shape> {
        throw UnsupportedOperationException("Not implemented")
    }

    public override fun initializeChildBlocks(manager: NDManager, dataType: DataType, vararg inputShapes: Shape) {
        val sub = manager.newSubManager()
            var X = sub.zeros(inputShapes[0], dataType)
            X = X.add(P[":, :{}, :", X.shape[1]])
            dropout.initialize(manager, dataType, X.shape)
        sub.close()
    }
}

In the positional embedding matrix $\mathbf{P}$,
rows correspond to positions within a sequence
and columns represent different positional encoding dimensions.
In the example below,
we can see that
the $6^{\mathrm{th}}$ and the $7^{\mathrm{th}}$
columns of the positional embedding matrix 
have a higher frequency than 
the $8^{\mathrm{th}}$ and the $9^{\mathrm{th}}$
columns.
The offset between 
the $6^{\mathrm{th}}$ and the $7^{\mathrm{th}}$ (same for the $8^{\mathrm{th}}$ and the $9^{\mathrm{th}}$) columns
is due to the alternation of sine and cosine functions.


In [6]:
    val encodingDim = 32
    val numSteps = 60
    val posEncoding = PositionalEncoding(encodingDim, 0f, 1000, manager)
    input = NDList(manager.zeros(Shape(1, numSteps.toLong(), encodingDim.toLong())))
    X = posEncoding.forward(ps, input, false)[0]
    var P = posEncoding.P[NDIndex(":, :{}, :", X.shape[1])]

val plotSize = 4
    val plotX = mutableListOf<IntArray>()
    val plotY = mutableListOf<FloatArray>()
    for (i in 0..3) {
        if (i == 0) {
            plotX.add(manager.arange(numSteps).toIntArray())
        } else {
            plotX.add(plotX[i - 1])
        }
        plotY.add(P[NDIndex("0, :, {},", i + 6)].toFloatArray())
    }

val labelSize = numSteps
var lossLabel = Array<String>(labelSize) { "Col6" }.toList() + Array<String>(labelSize) {"Col7"}.toList() + Array<String>(labelSize) {"Col8"}.toList() + Array<String>(labelSize) {"Col9"}.toList()

val data = mapOf( "epochCount" to plotX[0] + plotX[1] + plotX[2] + plotX[3],
                "loss" to plotY[0] + plotY[1] + plotY[2] + plotY[3],
                "lossLabel" to lossLabel)
var plot = letsPlot(data)
plot += geomLine { x = "epochCount" ; y = "loss" ; color = "lossLabel"}
plot + ggsize(500, 500)


<path d="M16.17354101786973 227.0 L16.17354101786973 227.0 L21.656097295113707 190.49588418548757 L27.138653572357683 155.1430946155028 L32.621209849601655 122.05662880160592 L38.103766126845635 92.280032212084 L43.586322404089614 66.75243980234322 L49.06887868133359 46.278975508429795 L54.551434958577566 31.505389170213192 L60.03399123582154 22.89760124683383 L65.51654751306552 20.727102442221224 L70.99910379030949 25.062343543226092 L76.48166006755346 35.766619779846906 L81.96421634479745 52.50231440500781 L87.44677262204141 74.7415629950437 L92.9293288992854 101.78299406441776 L98.41188517652938 132.77369112860072 L103.89444145377334 166.73622861233625 L109.37699773101733 202.59950545430183 L114.85955400826128 239.232310900634 L120.34211028550527 275.4793657199903 L125.82466656274926 310.19735573096705 L131.30722283999322 342.29138353737915 L136.7897791172372 370.74913703311574 L142.2723353944812 394.6731113954024 L147.75489167172515 413.30881043997675 L153.23744794896913 426.0683569149537 L158.7200042262131 432.54939795624125 L164.20256050345708 432.5475037206303 L169.68511678070107 426.06273570927704 L175.16767305794502 413.2996221672405 L180.650229335189 394.6607250625437 L186.13278561243297 370.73388474637807 L191.61534188967696 342.2737573060122 L197.09789816692094 310.17785371433604 L202.5804544441649 275.4586521305821 L208.0630107214089 239.2110891579227 L213.54556699865284 202.57839518243617 L219.02812327589683 166.71584712917155 L224.51067955314082 132.75473647225988 L229.99323583038478 101.7660689462315 L235.47579210762876 74.72720862518659 L240.95834838487275 52.49091209064832 L246.4409046621167 35.75861232930964 L251.9234609393607 25.057964660904645 L257.4060172166047 20.726462830196738 L262.88857349384864 22.90073780580002 L268.3711297710926 31.512178897857694 L273.8536860483366 46.28922160105273 L279.3362423255806 66.76590855555102 L284.8187986028246 92.29620701616463 L290.3013548800686 122.07499304684727 L295.7839111573125 155.16306404092097 L301.2664674345565 190.51683147928932 L306.7490237118005 227.02125972147704 L312.23157998904446 263.52502005750483 L317.71413626628845 298.87679485841227 L323.19669254353244 331.96178464456034 L328.67924882077637 361.7361425919966 L334.16180509802035 387.2610043504021 L339.64436137526434 407.7313074848869 " fill="none" stroke-width="1.6500000000000001" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M16.17354101786973 20.636363636363654 L16.17354101786973 20.636363636363654 L21.656097295113707 23.89067271622747 L27.138653572357683 33.55094222588974 L32.621209849601655 49.31252003799787 L38.103766126845635 70.67826770652425 L43.586322404089614 96.97434166344729 L49.06887868133359 127.37136806683108 L54.551434958577566 160.91062384301966 L60.03399123582154 196.53428603031418 L65.51654751306552 233.11884612590075 L70.99910379030949 269.5104202059182 L76.48166006755346 304.5612360239029 L81.96421634479745 337.16582797874105 L87.44677262204141 366.2958014119755 L92.9293288992854 391.0324766527523 L98.41188517652938 410.5956254330548 L103.89444145377334 424.36825585365295 L109.37699773101733 431.9159975485368 L114.85955400826128 433.00077954205597 L120.34211028550527 427.588394891132 L125.82466656274926 415.84954620491374 L131.30722283999322 398.15445814349437 L136.7897791172372 375.06126599962056 L142.2723353944812 347.2982910654762 L147.75489167172515 315.74108225107193 L153.23744794896913 281.38510342077774 L158.7200042262131 245.3138435063037 L164.20256050345708 208.66497857055882 L169.68511678070107 172.59429984201086 L175.16767305794502 138.23964328657496 L180.650229335189 106.68443940986288 L186.13278561243297 78.92392452196643 L191.61534188967696 55.833659833127825 L197.09789816692094 38.14186823368075 L202.5804544441649 26.406611214984565 L208.0630107214089 20.99796583435753 L213.54556699865284 22.086511698636173 L219.02812327589683 29.63796806335452 L224.51067955314082 43.414104049856036 L229.99323583038478 62.98045089028099 L235.47579210762876 87.719881

### Absolute Positional Information

To see how the monotonically decreased frequency
along the encoding dimension relates to absolute positional information,
let us print out the binary representations of $0, 1, \ldots, 7$.
As we can see,
the lowest bit, the second-lowest bit, and the third-lowest bit alternate on every number, every two numbers, and every four numbers, respectively.


In [7]:
    for (i in 0..7) {
        println(i.toString() + " in binary is " + Integer.toBinaryString(i))
    }


0 in binary is 0
1 in binary is 1
2 in binary is 10
3 in binary is 11
4 in binary is 100
5 in binary is 101
6 in binary is 110
7 in binary is 111


In binary representations,
a higher bit has a lower frequency than a lower bit.
Similarly,
as demonstrated in the heat map below,
the positional encoding decreases
frequencies along the encoding dimension
by using trigonometric functions.
Since the outputs are float numbers,
such continuous representations
are more space-efficient
than binary representations.


In [8]:
P = P.get(NDIndex("0, :, :")).expandDims(0).expandDims(0)
println(P)
val matrix = P.reshape(1,60,32).get(0)
println(matrix)
val seriesX = mutableListOf<Long>()
val seriesY = mutableListOf<Long>()
val seriesW = mutableListOf<Float>()
for(i in 0 until matrix.shape[0]) {
    val row = matrix.get(i)
    for(j in 0 until row.shape[0]) {
        seriesX.add(j)
        seriesY.add(i)
        seriesW.add(row.get(j).getFloat())
    }
}
val data = mapOf( "x" to seriesX, "y" to seriesY)
var plot = letsPlot(data)
plot += geomBin2D(drop=false, binWidth = kotlin.Pair(1,1), position = positionIdentity){x="x"; y = "y"; weight = seriesW }
plot += scaleFillGradient(low="blue", high="red")
//plot += scaleFillContinuous("red", "green")
plot + ggsize(500, 500)

ND: (1, 1, 60, 32) gpu(0) float32
Check the "Development Guideline"->Debug to enable array display.

ND: (60, 32) gpu(0) float32
Check the "Development Guideline"->Debug to enable array display.



<rect fill="#ff0002" stroke="#ffffff" stroke-width="0.0" x="150.30827396942857" y="110.06060606060606" height="6.878787878787875" width="11.05207896

### Relative Positional Information

Besides capturing absolute positional information,
the above positional encoding
also allows
a model to easily learn to attend by relative positions.
This is because
for any fixed position offset $\delta$,
the positional encoding at position $i + \delta$
can be represented by a linear projection
of that at position $i$.


This projection can be explained
mathematically.
Denoting
$\omega_j = 1/10000^{2j/d}$,
any pair of $(p_{i, 2j}, p_{i, 2j+1})$ 
in :eqref:`eq_positional-encoding-def`
can 
be linearly projected to $(p_{i+\delta, 2j}, p_{i+\delta, 2j+1})$
for any fixed offset $\delta$:

$$\begin{aligned}
&\begin{bmatrix} \cos(\delta \omega_j) & \sin(\delta \omega_j) \\  -\sin(\delta \omega_j) & \cos(\delta \omega_j) \\ \end{bmatrix}
\begin{bmatrix} p_{i, 2j} \\  p_{i, 2j+1} \\ \end{bmatrix}\\
=&\begin{bmatrix} \cos(\delta \omega_j) \sin(i \omega_j) + \sin(\delta \omega_j) \cos(i \omega_j) \\  -\sin(\delta \omega_j) \sin(i \omega_j) + \cos(\delta \omega_j) \cos(i \omega_j) \\ \end{bmatrix}\\
=&\begin{bmatrix} \sin\left((i+\delta) \omega_j\right) \\  \cos\left((i+\delta) \omega_j\right) \\ \end{bmatrix}\\
=& 
\begin{bmatrix} p_{i+\delta, 2j} \\  p_{i+\delta, 2j+1} \\ \end{bmatrix},
\end{aligned}$$

where the $2\times 2$ projection matrix does not depend on any position index $i$.

## Summary

* In self-attention, the queries, keys, and values all come from the same place.
* Both CNNs and self-attention enjoy parallel computation and self-attention has the shortest maximum path length. However, the quadratic computational complexity with respect to the sequence length makes self-attention prohibitively slow for very long sequences.
* To use the sequence order information, we can inject absolute or relative positional information by adding positional encoding to the input representations.

## Exercises

1. Suppose that we design a deep architecture to represent a sequence by stacking self-attention layers with positional encoding. What could be issues?
1. Can you design a learnable positional encoding method?
